# This is the second step in running corridor analysis

This needs to be run with a map open so the select at the end will work

Created by Amy Keister, last run by Amy Keister on 8/18/2022. It took 1 hour to run. 

In [1]:
import os
import arcpy
import numpy

In [2]:
import time
start = time.time()

In [3]:
# define spatial reference and workspaces
sr= arcpy.SpatialReference(5070)
#SourceWorkspace = 
OutWorkspace = r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\B_ConInlandHubs.gdb"

In [4]:
# define final outputs
Hub= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\Ba_Con_LM_Input.gdb\Hubs"

In [5]:
# define rasters used for cell size, extent, and snapping
SEraster= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintExtent2022.tif"

In [6]:
# define additional inputs
TNCLocalCon= r"F:\GIS_DATA\DecisionSupportTools\TNCTerrestrialResilient\Local_Connectedness\Local_connectedness_CONUSnew.tif"
Zonation= r"D:\SE_Blueprint_2023\7_CombineZonation\CombineZonation.tif"
Inland= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\A_ConInlandResistance.gdb\InlandNearShoreBuffRNoEst"
Inlandv= r"D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\A_ConInlandResistance.gdb\InlandNearShore_Buff2"
PAD= r"F:\GIS_DATA\ProtectedLands\PAD_US2_1_GDB\PAD_US2_1.gdb\PADUS2_1Combined_Fee_Easement"
res= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Ancillary_Data_Download\Reservoirs.tif"
Extent= r"F:\GIS_DATA\SECAS\SE_Blueprint_2022\Southeast_Blueprint_2022_Data_Download\SEBlueprint20221215\Inputs\BaseBlueprint\1_ExtentLayers\BaseBlueprintSubRgn.shp"

# Start Analysis

In [7]:
# Change the workspace to where I am saving the outputs
arcpy.env.workspace = OutWorkspace

In [8]:
# Print the current workspace to make sure I'm in the right spot
print(arcpy.env.workspace)

D:\SE_Blueprint_2023\8_PrepLinkageMapperRuns\B_ConInlandHubs.gdb


# Identify areas in the top 10% of the zonation runs

In [9]:
# use a conditional statement to pull out the top 10%, use environemnts to clip to inland
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Inland, snapRaster=Inland, cellSize=Inland, mask=Inland):
    out_raster = arcpy.sa.Con(Zonation, 1, 0, "VALUE >90"); out_raster.save("Top10")

In [10]:
# reclassify to turn NoData into 0
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
out_raster = arcpy.sa.Reclassify("Top10", "Value", "0 0;1 1;NODATA 0", "DATA"); out_raster.save("Top10z")

# Bring in Protected Areas
In the South Atlantic we used TNC's secured areas, and we had it pretty figured out what to keep and remove. I'm learning on 
PADUS 2.1 Combined Fee, Easement, but we need to keep an eye out for improvements
I checked and it doesn't seem like this layer includes department of defense lands like Army or Air force bases, so no need to remove them

In [11]:
# Bring in PADUS 2.1 Combined Fee, Easement 
# remove the weird squares 
arcpy.analysis.Select(PAD, "PAD1", "Loc_Ds NOT IN ('School Trust Land','School Lands','State Land Board','3201')")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\B_ConInlandHubs.gdb\\PAD1'>

In [12]:
# add field to use to convert to raster
arcpy.management.CalculateField("PAD1", "raster", "1", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\B_ConInlandHubs.gdb\\PAD1'>

In [13]:
# convert to raster
with arcpy.EnvManager(outputCoordinateSystem=sr, snapRaster=SEraster, cellSize=SEraster, extent=SEraster):
    arcpy.conversion.PolygonToRaster("PAD1", "raster", "PAD1a", "CELL_CENTER", "NONE", SEraster)

# Identify potential hubs from local connectedness
Areas with a local connected score >= 1 standard deviation were considered potential hubs outside of the South Atlantic area

In [14]:
# reclassify Inland near shore raster to seperate buffered area from the area covered by zonation
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Inland, snapRaster=Inland, cellSize=Inland):
    out_raster = arcpy.sa.Reclassify(SEraster, "Value", "1 0;NODATA NODATA", "DATA"); out_raster.save("SEraster1")

In [15]:
# combine rasters, keeping min value
out_raster = arcpy.sa.CellStatistics(["SEraster1", Inland],\
"MINIMUM", "DATA", "SINGLE_BAND"); out_raster.save("Buff")

In [16]:
# clip local connectedness to buffer area
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Inland, snapRaster=Inland, cellSize=Inland):
    out_raster = arcpy.sa.Times(TNCLocalCon, "Buff"); out_raster.save("TNCLocalConB")

In [17]:
# pull out high areas in local connectedness
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Inland, snapRaster=Inland, cellSize=Inland, mask=Inland):
    out_raster = arcpy.sa.Con("TNCLocalConB", 1, 0, "VALUE >1000"); out_raster.save("TNChigh")

# Combine 3 potential hub layers 

In [18]:
# combine rasters, keeping max value
out_raster = arcpy.sa.CellStatistics(["Top10z", "PAD1a", "TNChigh"],\
"MAXIMUM", "DATA", "SINGLE_BAND"); out_raster.save("PotHub")

# Remove reservoirs as potential hubs

In [19]:
# reclassify reservoirs
# 0 1 is <=1 (1 is included)
# 1 5 is >1 - 5 (1 is not included, 5 is included)
# 5 20 is >5 - 20 (5 is not included, 20 is included)
with arcpy.EnvManager(outputCoordinateSystem=sr, extent=Inland, snapRaster=Inland, cellSize=Inland):
    out_raster = arcpy.sa.Reclassify(res, "Value", "0 0;1 1;NODATA 1", "DATA"); out_raster.save("res1")

In [20]:
# remove reservoirs as potential hubs from the South Atlantic Area
out_raster = arcpy.sa.Times("res1", "PotHub"); out_raster.save("PotHub2")

# convert to 90 meter raster and back to vector to make sure it won't throw errors in corridor analysis
Because we need to do corridor analysis at 90 meters, we need to make sure that after cores are converted to 90 meter rasters, there aren't small isolated cores

In [21]:
# resample to 90 meters using default esri method
arcpy.management.Resample("PotHub2", "PotHub2_90", "90 90", '')

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\B_ConInlandHubs.gdb\\PotHub2_90'>

# Calculate size of potential hubs

In [22]:
# use region group to create clumps of isolated potential hubs
out_raster = arcpy.sa.RegionGroup("PotHub2_90", "EIGHT", '', '', '0'); out_raster.save("PotHub3a")

In [23]:
# pull out clumps that are larger than 2,000 ha (5,000 acres), keep their region group value
# this is for the 30 meter resolution version
#out_raster = arcpy.sa.Con("PotHub3a", "PotHub3a", '', "Count > 22482 And Link = 1"); out_raster.save("PotHub4")

In [24]:
# pull out clumps that are larger than 2,000 ha (5,000 acres), keep their region group value
# this is for the 90 meter resolution version
out_raster = arcpy.sa.Con("PotHub3a", "PotHub3a", '', "Count > 2498 And Link = 1"); out_raster.save("PotHub4")

In [25]:
# convert to vector
arcpy.conversion.RasterToPolygon("PotHub4", "PotHub5", "NO_SIMPLIFY", "Value", "MULTIPLE_OUTER_PART", None)

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\B_ConInlandHubs.gdb\\PotHub5'>

In [31]:
#Select potential hubs that intersect buffered Southeast inland area
arcpy.management.SelectLayerByLocation("PotHub5", "INTERSECT", Inlandv, None, "NEW_SELECTION", "NOT_INVERT")

<Result 'PotHub5'>

In [32]:
#Add field that ESRI is willing to read
arcpy.management.CalculateField("PotHub5", "UID", "!OBJECTID!", "PYTHON3", '', "SHORT", "NO_ENFORCE_DOMAINS")

<Result 'PotHub5'>

In [33]:
#I'm going to clip the hubs to the buffer area because having hubs with centroids that don't have resistance is 
# throwing an error in linkage mapper
# I'm not sure if this will work because it will make some hubs on the edge split into multiple polygons
#arcpy.analysis.Clip(
#    in_features="PotHub5",
#    clip_features=Inlandv,
#    out_feature_class=Hub,
#    cluster_tolerance=None
#)

In [34]:
# copy features
arcpy.management.CopyFeatures("PotHub5", Hub, '', None, None, None)

<Result 'D:\\SE_Blueprint_2023\\8_PrepLinkageMapperRuns\\Ba_Con_LM_Input.gdb\\Hubs'>

In [30]:
end = time.time()
print(end - start)

2846.573084592819
